<a href="https://colab.research.google.com/github/hagarbarakat/Emotion_Detection/blob/master/Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import csv
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import struct as st
import os
import scipy.io
import math
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
import shutil
from sklearn.model_selection import train_test_split
import cv2
from keras.models import Sequential 
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [0]:
Training_x = [] #training data list
Training_y = [] #training labels list
Validation_x = [] #validation data list
Validation_y = []  #validation label list


with open('fer2013.csv') as csvin:
    data=csv.reader(csvin)
    for row in data:
      #row[-1] last item which indicates the usage of each image (training, public test, private test)
        if row[-1] == 'Training' or row[-1] == 'PublicTest':                  #Training and Public test images are used for training
            temp_list = []    
            for pixel in row[1].split( ): 
                temp_list.append(int(pixel))
            I = np.asarray(temp_list).reshape(48,48,1) #reshape data (48,48)
            Training_y.append(int(row[0]))
            #y_training=np.array(Training_y)
            y_training=pd.get_dummies(Training_y) #hot encoding for training labels
            Training_x.append(I.tolist())

        if row[-1] == 'PrivateTest':           #Privates Test images are used for validation
            temp_list = []
            for pixel in row[1].split( ):
                temp_list.append(int(pixel))
            I = np.asarray(temp_list).reshape(48,48,1)
            Validation_y.append(int(row[0]))
            y_validation=pd.get_dummies(Validation_y) #hot encoding for validation labels
            Validation_x.append(I.tolist())
  
print(np.shape(Training_x))
print(np.shape(Training_y))
print(np.shape(y_training))
print(np.shape(Validation_x))
 


(32298, 48, 48, 1)
(32298,)
(32298, 7)
(3589, 48, 48, 1)


In [0]:
keras.optimizers.Adam(lr=0.003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [0]:
#initialize cnn model
model= Sequential()

#Convolution
model.add(Convolution2D(filters = 64, kernel_size = (2, 2),strides=(2,2), input_shape = (48, 48, 1), activation = 'sigmoid'))
model.add(Dropout(0.35))
model.add(Convolution2D(filters = 128, kernel_size = (2, 2),strides=(2,2), input_shape = (48, 48, 1), activation = 'sigmoid'))
model.add(Dropout(0.35))
model.add(Convolution2D(filters = 256, kernel_size = (2, 2),strides=(2,2), input_shape = (48, 48, 1), activation = 'sigmoid'))


#Pooling

model.add(MaxPooling2D(pool_size = (2, 2), strides=(2,2)))



#Flattening
model.add(Flatten())
model.add(Dropout(0.35))

#Fullconnection

model.add(Dense(activation='sigmoid', units=256))

model.add(Dropout(0.35))

model.add(Dense(activation='softmax',units=7))


#Compiling CNN
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 24, 24, 64)        320       
_________________________________________________________________
dropout_21 (Dropout)         (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 12, 12, 128)       32896     
_________________________________________________________________
dropout_22 (Dropout)         (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 6, 6, 256)         131328    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 2304)             

In [0]:
# train the network
model.fit(np.array(Training_x), y_training, epochs=50, validation_data=(np.array(Validation_x), y_validation),batch_size=64)

Train on 32298 samples, validate on 3589 samples
Epoch 1/50
32298/32298 [==============================] - 9s 290us/step - loss: 1.8497 - acc: 0.2339 - val_loss: 1.8156 - val_acc: 0.2449
Epoch 2/50
32298/32298 [==============================] - 8s 252us/step - loss: 1.8170 - acc: 0.2507 - val_loss: 1.8163 - val_acc: 0.2449
Epoch 3/50
32298/32298 [==============================] - 8s 256us/step - loss: 1.8039 - acc: 0.2531 - val_loss: 1.7791 - val_acc: 0.2583
Epoch 4/50
32298/32298 [==============================] - 8s 260us/step - loss: 1.7656 - acc: 0.2765 - val_loss: 1.7540 - val_acc: 0.2800
Epoch 5/50
32298/32298 [==============================] - 8s 253us/step - loss: 1.7285 - acc: 0.3019 - val_loss: 1.6876 - val_acc: 0.3377
Epoch 6/50
32298/32298 [==============================] - 8s 253us/step - loss: 1.6889 - acc: 0.3232 - val_loss: 1.6453 - val_acc: 0.3533
Epoch 7/50
32298/32298 [==============================] - 8s 254us/step - loss: 1.6672 - acc: 0.3365 - val_loss: 1.6285 - v